# Comparing ADAM to MCMC

This notebook aims to illustrate the existing functionality in BoTorch for different methods of handeling the hyperparameter optimization of a gaussian process surrogate model; for bayesian optimization. For brevity discussion of different surrogate models is ommited, but both methods are generally suitable for hyperparameter optimization of multiple different surrogate models.

In Bayesian Optimization for an arbitrary acquisition function we wish to find the $arg\:max_{x \in \mathbb{X}} \alpha(x \mid \theta)$. We can and often do choose $\theta$ through an optimizer (e.g. ADAM). Alternatively we can consider the distribution of $\theta$ such that we now wish to find.
$$
arg\:max_{x\in\mathcal{X}} \int_\Theta \alpha(x\mid\theta)P(\theta)\,d\theta
\tag{1}
$$
This integral is of course intractable, a very popular way of approximating intractable integrals is monte carlo methods the most commonly used monte carlo methos is Markov Chain Monte Carlo (MCMC). 

The conclusion which we aim to draw in the remainder of the notebook is that the approximation of (1) through MCMC leads to better evaluations of an objective function than choosing $\theta$ to be a single value found through an optimizer. This conclusion is one that is supported in the literature.

We begin with defining a dataset class for readability the specifics of the class are not important, it just handels the storing of evaluations of the target (test/toy) functions.

## Preamble Code

In [ ]:
import torch
from tests.target_functions import BaseTarget

class Dataset():
    def __init__(
            self, 
            target: BaseTarget,
        ) -> None:
        self.target = target
        self.X: torch.Tensor | None = None
        self.y: torch.Tensor | None = None
        return

    def random_evals(
            self, 
            seed: int, 
            n_iters: int
        ) -> None:
        assert self.X is None and self.y is None
        random_gen = torch.Generator().manual_seed(seed)
        rand_vals = torch.rand(size = (n_iters, self.target.dim), 
                               generator=random_gen)
        print(self.target.bounds)
        X = self.target.bounds[0] + (self.target.bounds[1]-self.target.bounds[0]) * rand_vals
        y = torch.tensor([self.target.sample(x) for x in X])
        self.X = X
        self.y = y.reshape(-1, 1)

    def eval_x(
            self,
            x_star: torch.Tensor,
    ) -> None:
        y_star = self.target.sample(x_star)
        self.X = torch.cat((self.X, x_star.unsqueeze(0)))
        self.y = torch.cat((self.y, y_star.reshape((-1,1))))
        return

For the target functions used visit `tests > target_functions` for declarations, or see [botorch docs](https://botorch.readthedocs.io/en/latest/test_functions.html) for information on the target funcitons, the dimensions used for each will be indicated on any plots generated.

## Main Code 

In [8]:
MODEL_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from botorch.models import SingleTaskGP, SaasFullyBayesianSingleTaskGP

from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll, fit_fully_bayesian_model_nuts, ExactMarginalLogLikelihood
from botorch.acquisition import LogExpectedImprovement
from botorch.optim import optimize_acqf

from tqdm import tqdm
import pickle

import os

def ADAM_BO(
        target: BaseTarget,
        seed: int,
        n_random_evals: int,
        n_bo_evals: int,
        disable_prog_bar: bool = True,
        save_results: bool = True
    ) -> dict:
    assert target.num_evals == 0, 'target must not have been evaluated'
    
    dataset = Dataset(target)
    dataset.random_evals(seed, n_random_evals)

    for _ in tqdm(range(n_bo_evals), 
                  disable = disable_prog_bar):
        gp = SingleTaskGP(train_X=dataset.X,
                          train_Y=dataset.y,
                          input_transform=Normalize(d=target.dim),
                          outcome_transform=Standardize(m=1),)
        gp.to(MODEL_DEVICE)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        fit_gpytorch_mll(mll)
        logEI = LogExpectedImprovement(model=gp, best_f=dataset.y.max())
        x_star, acq_val = optimize_acqf(logEI, bounds=target.bounds, q=1, num_restarts=5, raw_samples=20)
        x_star = x_star[0]
        dataset.eval_x(x_star)

    results = target.get_results()

    if save_results:
        save_name = f'data/notebook_plot_data/{target.target_name}_ADAM.pkl'

        with open(save_name, "wb") as f:
            pickle.dump(results, f)

    return results

def MCMC_BO(
        target: BaseTarget,
        seed: int,
        n_random_evals: int,
        n_bo_evals: int,
        warm_up_steps: int,
        num_samples: int,
        thinning: int,
        disable_prog_bar: bool = True,
        save_results: bool = True,
    ) -> dict:
    assert target.num_evals == 0, 'target must not have been evaluated'
    
    dataset = Dataset(target)
    dataset.random_evals(seed, n_random_evals)

    for _ in tqdm(range(n_bo_evals), 
                  disable = disable_prog_bar):
        model = SaasFullyBayesianSingleTaskGP(
            train_X=dataset.X,
            train_Y=dataset.y,
            input_transform=Normalize(d=target.dim),
            outcome_transform=Standardize(m=1),)
        model.to(MODEL_DEVICE)
        fit_fully_bayesian_model_nuts(
            model = model,
            warmup_steps=warm_up_steps,
            num_samples=num_samples,
            thinning=thinning,
            disable_progbar=True,
        )
        logEI = LogExpectedImprovement(model=model, best_f=dataset.y.max())
        
        x_star, acq_val = optimize_acqf(logEI, bounds=target.bounds, q=1, num_restarts=5, raw_samples=20)
        x_star = x_star[0]
        dataset.eval_x(x_star)

    results = target.get_results()

    if save_results:
        save_name = f'data/notebook_plot_data/{target.target_name}_MCMC.pkl'

        with open(save_name, "wb") as f:
            pickle.dump(results, f)

    return results

## Test Running

In [14]:
import time
from tests.target_functions import Ackley, Branin, Hartmann, Rosenbrock

N_RANDOM_EVALS = 5
N_BO_EVALS = 1
SEED = 1

target_classes: list[BaseTarget] = [Ackley, Branin, Hartmann, Rosenbrock]
test_fns: list[callable] = [ADAM_BO, MCMC_BO]
# arguments common accross both test functions
common_args: dict = {'n_random_evals': N_RANDOM_EVALS, 'n_bo_evals': N_BO_EVALS, 'seed': SEED}

# MCMC takes extra args the number of MCMC iterations is warm_up_steps + num_samples
# a thinning of 1 means all of num_samples are used to infer P(\theta)
test_args: list[dict] = [{}, {'warm_up_steps': 256, 'num_samples': 128, 'thinning': 1,}]

results_text = []
for target_class in target_classes:
    for i, test_fn in enumerate(test_fns):
        target: BaseTarget = target_class()
        start_time = time.time()
        test_fn(target = target, **common_args, **(test_args[i]))
        end_time = time.time()
        
        text_line = (f'{test_fn.__name__} took {end_time-start_time} seconds to run on the {target.dim}',
                    f' dimensional {target.target_name} synthetic test function')
        results_text.append(text_line)
        print(text_line)

IndexError: index 1 is out of bounds for dimension 0 with size 1

## Plotting